# Day15_0: 앙상블 & AutoML (Ensemble & AutoML)

## 학습 목표

**Part 1: 고급 앙상블**
1. Voting (Hard/Soft) 앙상블 이해하기
2. Stacking 심화 기법 익히기
3. Blending 전략 적용하기
4. Plotly로 앙상블 모델 비교하기
5. 최적 앙상블 조합 찾기

**Part 2: AutoML & 모델 해석**
1. Optuna를 활용한 하이퍼파라미터 최적화
2. Feature Importance 분석하기
3. SHAP Values로 모델 해석하기
4. Partial Dependence Plot 그리기

---

## 왜 이것을 배우나요?

| 개념 | 비즈니스 활용 | 실무 예시 |
|------|-------------|----------|
| Voting Ensemble | 여러 모델 의견 종합 | 다수결로 안정적 예측 |
| Stacking | 모델 결과를 특성으로 활용 | Kaggle 상위권 필수 기법 |
| Optuna | 자동 하이퍼파라미터 튜닝 | 수작업 튜닝 대비 5배 효율 |
| SHAP | 예측 근거 설명 | 고객/경영진 설득, 규제 대응 |

**분석가 관점**: Kaggle 미니 대회 마무리! 앙상블로 성능을 끌어올리고, SHAP로 모델을 설명합니다.

---

# Part 1: 고급 앙상블

---

## 1.1 앙상블 개요

### 앙상블이란?

**앙상블(Ensemble)**: 여러 모델의 예측을 결합하여 더 좋은 성능을 얻는 기법

```
앙상블 종류
├── Bagging: 같은 알고리즘, 다른 데이터 (Random Forest)
├── Boosting: 순차적 학습, 오류 보정 (XGBoost, LightGBM)
├── Voting: 다른 알고리즘, 투표 결합
├── Stacking: 메타 모델로 결합
└── Blending: 홀드아웃 기반 스태킹
```

In [ ]:
# 필요한 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, 
    GradientBoostingClassifier,
    VotingClassifier, 
    StackingClassifier
)
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, 
    roc_auc_score, confusion_matrix
)
from sklearn.datasets import make_classification

import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

In [ ]:
# 실습용 데이터셋 생성: 고객 이탈 예측
np.random.seed(42)

X, y = make_classification(
    n_samples=1000,
    n_features=10,
    n_informative=6,
    n_redundant=2,
    n_classes=2,
    weights=[0.7, 0.3],  # 불균형 데이터
    random_state=42
)

# 특성명 부여
feature_names = [
    'tenure', 'monthly_charge', 'total_charge', 'age',
    'complaints', 'support_calls', 'contract_length',
    'payment_delay', 'usage_score', 'satisfaction'
]

df = pd.DataFrame(X, columns=feature_names)
df['churned'] = y

print(f"데이터셋 크기: {df.shape}")
print(f"\n이탈 비율: {df['churned'].mean():.1%}")
print(f"\n데이터 미리보기:")
df.head()

In [ ]:
# 데이터 분할
X = df.drop('churned', axis=1)
y = df['churned']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"훈련 데이터: {X_train.shape[0]}개")
print(f"테스트 데이터: {X_test.shape[0]}개")

---

## 1.2 Voting Ensemble

### Hard Voting vs Soft Voting

| 방식 | 설명 | 계산 |
|-----|------|-----|
| Hard Voting | 다수결 투표 | 가장 많이 예측된 클래스 |
| Soft Voting | 확률 평균 | 평균 확률이 높은 클래스 |

```
예시: 3개 모델 예측
- 모델A: 클래스1 (확률 0.9)
- 모델B: 클래스0 (확률 0.6)
- 모델C: 클래스1 (확률 0.7)

Hard Voting: 클래스1 (2:1)
Soft Voting: 클래스1 평균 = (0.9 + 0.4 + 0.7) / 3 = 0.67
             클래스0 평균 = (0.1 + 0.6 + 0.3) / 3 = 0.33
             -> 클래스1
```

In [ ]:
# 개별 모델 정의
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'SVM': SVC(probability=True, random_state=42)
}

# 개별 모델 성능 평가
individual_results = []

for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)
    
    individual_results.append({
        'Model': name,
        'Accuracy': accuracy,
        'AUC': auc
    })
    print(f"{name}: Accuracy={accuracy:.4f}, AUC={auc:.4f}")

results_df = pd.DataFrame(individual_results)

In [ ]:
# Hard Voting
hard_voting = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
    ],
    voting='hard'
)

hard_voting.fit(X_train_scaled, y_train)
y_pred_hard = hard_voting.predict(X_test_scaled)
accuracy_hard = accuracy_score(y_test, y_pred_hard)

print(f"Hard Voting Accuracy: {accuracy_hard:.4f}")

In [ ]:
# Soft Voting
soft_voting = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
    ],
    voting='soft'
)

soft_voting.fit(X_train_scaled, y_train)
y_pred_soft = soft_voting.predict(X_test_scaled)
y_proba_soft = soft_voting.predict_proba(X_test_scaled)[:, 1]

accuracy_soft = accuracy_score(y_test, y_pred_soft)
auc_soft = roc_auc_score(y_test, y_proba_soft)

print(f"Soft Voting Accuracy: {accuracy_soft:.4f}")
print(f"Soft Voting AUC: {auc_soft:.4f}")

### 실무 예시: 가중 Soft Voting

성능이 좋은 모델에 더 높은 가중치를 부여합니다.

In [ ]:
# 가중 Soft Voting (성능 좋은 모델에 높은 가중치)
weighted_voting = VotingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
    ],
    voting='soft',
    weights=[1, 2, 3]  # GB에 가장 높은 가중치
)

weighted_voting.fit(X_train_scaled, y_train)
y_pred_weighted = weighted_voting.predict(X_test_scaled)
y_proba_weighted = weighted_voting.predict_proba(X_test_scaled)[:, 1]

accuracy_weighted = accuracy_score(y_test, y_pred_weighted)
auc_weighted = roc_auc_score(y_test, y_proba_weighted)

print(f"Weighted Soft Voting Accuracy: {accuracy_weighted:.4f}")
print(f"Weighted Soft Voting AUC: {auc_weighted:.4f}")

---

## 1.3 Stacking

### Stacking 구조

```
Level 0 (Base Models):
  Model1 -> pred1
  Model2 -> pred2
  Model3 -> pred3
      ↓
Level 1 (Meta Model):
  [pred1, pred2, pred3] -> Final Prediction
```

**핵심**: 베이스 모델의 예측을 새로운 특성으로 사용하여 메타 모델 학습

In [ ]:
# Stacking Classifier
stacking = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),
        ('knn', KNeighborsClassifier(n_neighbors=5))
    ],
    final_estimator=LogisticRegression(random_state=42),
    cv=5,  # 5-fold CV로 메타 특성 생성
    stack_method='predict_proba',  # 확률 사용
    passthrough=False  # 원본 특성 포함 여부
)

stacking.fit(X_train_scaled, y_train)
y_pred_stack = stacking.predict(X_test_scaled)
y_proba_stack = stacking.predict_proba(X_test_scaled)[:, 1]

accuracy_stack = accuracy_score(y_test, y_pred_stack)
auc_stack = roc_auc_score(y_test, y_proba_stack)

print(f"Stacking Accuracy: {accuracy_stack:.4f}")
print(f"Stacking AUC: {auc_stack:.4f}")

In [ ]:
# Stacking + 원본 특성 (passthrough=True)
stacking_passthrough = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression(random_state=42, max_iter=1000)),
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
    ],
    final_estimator=RandomForestClassifier(n_estimators=50, random_state=42),
    cv=5,
    passthrough=True  # 원본 특성 포함
)

stacking_passthrough.fit(X_train_scaled, y_train)
y_pred_stack_pt = stacking_passthrough.predict(X_test_scaled)
y_proba_stack_pt = stacking_passthrough.predict_proba(X_test_scaled)[:, 1]

accuracy_stack_pt = accuracy_score(y_test, y_pred_stack_pt)
auc_stack_pt = roc_auc_score(y_test, y_proba_stack_pt)

print(f"Stacking (passthrough) Accuracy: {accuracy_stack_pt:.4f}")
print(f"Stacking (passthrough) AUC: {auc_stack_pt:.4f}")

---

## 1.4 Blending

### Blending vs Stacking

| 구분 | Stacking | Blending |
|-----|----------|----------|
| 메타 특성 생성 | K-Fold CV | Holdout 세트 |
| 장점 | 모든 데이터 활용 | 구현 간단 |
| 단점 | 느림, 복잡 | 데이터 손실 |

In [ ]:
# Blending 구현
# Step 1: 훈련 데이터를 train/blend로 분할
X_train_base, X_blend, y_train_base, y_blend = train_test_split(
    X_train_scaled, y_train, test_size=0.3, random_state=42, stratify=y_train
)

print(f"Base 훈련: {X_train_base.shape[0]}개")
print(f"Blend 세트: {X_blend.shape[0]}개")

In [ ]:
# Step 2: 베이스 모델 훈련 및 Blend 세트 예측
base_models = [
    ('lr', LogisticRegression(random_state=42, max_iter=1000)),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42))
]

blend_features = []
test_features = []

for name, model in base_models:
    model.fit(X_train_base, y_train_base)
    
    # Blend 세트 예측
    blend_pred = model.predict_proba(X_blend)[:, 1]
    blend_features.append(blend_pred)
    
    # 테스트 세트 예측
    test_pred = model.predict_proba(X_test_scaled)[:, 1]
    test_features.append(test_pred)
    
    print(f"{name} 훈련 완료")

# 메타 특성 행렬 생성
X_blend_meta = np.column_stack(blend_features)
X_test_meta = np.column_stack(test_features)

print(f"\nBlend 메타 특성: {X_blend_meta.shape}")
print(f"Test 메타 특성: {X_test_meta.shape}")

In [ ]:
# Step 3: 메타 모델 훈련
meta_model = LogisticRegression(random_state=42)
meta_model.fit(X_blend_meta, y_blend)

# 최종 예측
y_pred_blend = meta_model.predict(X_test_meta)
y_proba_blend = meta_model.predict_proba(X_test_meta)[:, 1]

accuracy_blend = accuracy_score(y_test, y_pred_blend)
auc_blend = roc_auc_score(y_test, y_proba_blend)

print(f"Blending Accuracy: {accuracy_blend:.4f}")
print(f"Blending AUC: {auc_blend:.4f}")

---

## 1.5 Plotly로 앙상블 모델 비교

In [ ]:
# 모든 모델 성능 비교
all_results = [
    {'Model': 'Logistic Regression', 'Type': 'Individual', 'AUC': results_df[results_df['Model']=='Logistic Regression']['AUC'].values[0]},
    {'Model': 'Random Forest', 'Type': 'Individual', 'AUC': results_df[results_df['Model']=='Random Forest']['AUC'].values[0]},
    {'Model': 'Gradient Boosting', 'Type': 'Individual', 'AUC': results_df[results_df['Model']=='Gradient Boosting']['AUC'].values[0]},
    {'Model': 'Hard Voting', 'Type': 'Ensemble', 'AUC': accuracy_hard},  # Hard voting은 확률 없음
    {'Model': 'Soft Voting', 'Type': 'Ensemble', 'AUC': auc_soft},
    {'Model': 'Weighted Voting', 'Type': 'Ensemble', 'AUC': auc_weighted},
    {'Model': 'Stacking', 'Type': 'Ensemble', 'AUC': auc_stack},
    {'Model': 'Stacking (passthrough)', 'Type': 'Ensemble', 'AUC': auc_stack_pt},
    {'Model': 'Blending', 'Type': 'Ensemble', 'AUC': auc_blend}
]

comparison_df = pd.DataFrame(all_results)
comparison_df = comparison_df.sort_values('AUC', ascending=True)

# Plotly 시각화
fig = px.bar(
    comparison_df,
    x='AUC',
    y='Model',
    color='Type',
    orientation='h',
    title='개별 모델 vs 앙상블 모델 AUC 비교',
    color_discrete_map={'Individual': '#636EFA', 'Ensemble': '#EF553B'}
)

fig.update_layout(
    xaxis_title='AUC Score',
    yaxis_title='Model',
    height=500
)

fig.show()

In [ ]:
# 레이더 차트로 앙상블 비교
ensemble_metrics = pd.DataFrame({
    'Metric': ['AUC', 'Accuracy', 'Training Speed', 'Interpretability', 'Stability'],
    'Voting': [auc_soft, accuracy_soft, 0.9, 0.7, 0.8],
    'Stacking': [auc_stack, accuracy_stack, 0.5, 0.4, 0.9],
    'Blending': [auc_blend, accuracy_blend, 0.7, 0.5, 0.7]
})

fig = go.Figure()

for method in ['Voting', 'Stacking', 'Blending']:
    fig.add_trace(go.Scatterpolar(
        r=ensemble_metrics[method].tolist() + [ensemble_metrics[method].iloc[0]],
        theta=ensemble_metrics['Metric'].tolist() + [ensemble_metrics['Metric'].iloc[0]],
        fill='toself',
        name=method
    ))

fig.update_layout(
    polar=dict(radialaxis=dict(visible=True, range=[0, 1])),
    showlegend=True,
    title='앙상블 방법 비교 (레이더 차트)',
    height=500
)

fig.show()

---

# Part 2: AutoML & 모델 해석

---

## 2.1 Optuna를 활용한 하이퍼파라미터 최적화

### Optuna란?

- 자동 하이퍼파라미터 최적화 프레임워크
- 베이지안 최적화 기반
- 조기 종료(Pruning) 지원
- 시각화 도구 내장

In [ ]:
# Optuna 설치 확인 및 임포트
try:
    import optuna
    from optuna.visualization import plot_optimization_history, plot_param_importances
    print(f"Optuna 버전: {optuna.__version__}")
except ImportError:
    print("Optuna 설치가 필요합니다: pip install optuna")

In [ ]:
# Optuna로 Random Forest 튜닝
import optuna
from sklearn.model_selection import cross_val_score

def objective_rf(trial):
    """Random Forest 최적화 목적 함수"""
    
    # 하이퍼파라미터 탐색 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    }
    
    model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
    
    # 5-Fold CV로 AUC 계산
    scores = cross_val_score(model, X_train_scaled, y_train, 
                            cv=5, scoring='roc_auc', n_jobs=-1)
    
    return scores.mean()

# 최적화 실행
optuna.logging.set_verbosity(optuna.logging.WARNING)

study_rf = optuna.create_study(direction='maximize', study_name='rf_optimization')
study_rf.optimize(objective_rf, n_trials=30, show_progress_bar=True)

print(f"\n최적 AUC: {study_rf.best_value:.4f}")
print(f"최적 파라미터: {study_rf.best_params}")

In [ ]:
# 최적 모델로 재학습
best_rf = RandomForestClassifier(**study_rf.best_params, random_state=42)
best_rf.fit(X_train_scaled, y_train)

y_pred_best_rf = best_rf.predict(X_test_scaled)
y_proba_best_rf = best_rf.predict_proba(X_test_scaled)[:, 1]

print(f"최적화된 RF Accuracy: {accuracy_score(y_test, y_pred_best_rf):.4f}")
print(f"최적화된 RF AUC: {roc_auc_score(y_test, y_proba_best_rf):.4f}")

In [ ]:
# Optuna 최적화 과정 시각화 (Plotly)
trials_df = study_rf.trials_dataframe()

fig = px.line(
    trials_df,
    x='number',
    y='value',
    title='Optuna 최적화 과정',
    labels={'number': 'Trial', 'value': 'AUC Score'}
)

# 최적점 표시
best_trial = trials_df.loc[trials_df['value'].idxmax()]
fig.add_scatter(
    x=[best_trial['number']],
    y=[best_trial['value']],
    mode='markers',
    marker=dict(size=15, color='red', symbol='star'),
    name=f'Best: {best_trial["value"]:.4f}'
)

fig.update_layout(height=400)
fig.show()

In [ ]:
# 파라미터 중요도 시각화
param_importance = optuna.importance.get_param_importances(study_rf)

importance_df = pd.DataFrame([
    {'Parameter': k, 'Importance': v}
    for k, v in param_importance.items()
]).sort_values('Importance', ascending=True)

fig = px.bar(
    importance_df,
    x='Importance',
    y='Parameter',
    orientation='h',
    title='하이퍼파라미터 중요도 (Optuna)',
    color='Importance',
    color_continuous_scale='Viridis'
)

fig.update_layout(height=400)
fig.show()

---

## 2.2 Feature Importance 분석

In [ ]:
# Feature Importance (Random Forest 기반)
feature_importance = pd.DataFrame({
    'Feature': feature_names,
    'Importance': best_rf.feature_importances_
}).sort_values('Importance', ascending=True)

fig = px.bar(
    feature_importance,
    x='Importance',
    y='Feature',
    orientation='h',
    title='Feature Importance (Random Forest)',
    color='Importance',
    color_continuous_scale='Blues'
)

fig.update_layout(height=500)
fig.show()

In [ ]:
# Permutation Importance
from sklearn.inspection import permutation_importance

perm_importance = permutation_importance(
    best_rf, X_test_scaled, y_test,
    n_repeats=10, random_state=42, n_jobs=-1
)

perm_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': perm_importance.importances_mean,
    'Std': perm_importance.importances_std
}).sort_values('Importance', ascending=True)

fig = px.bar(
    perm_df,
    x='Importance',
    y='Feature',
    orientation='h',
    title='Permutation Importance',
    error_x='Std',
    color='Importance',
    color_continuous_scale='Reds'
)

fig.update_layout(height=500)
fig.show()

---

## 2.3 SHAP Values

### SHAP이란?

**SHAP (SHapley Additive exPlanations)**: 게임 이론의 Shapley Value를 활용한 모델 해석 기법

- 각 특성이 예측에 기여한 정도를 정량화
- 개별 예측과 전체 모델 모두 해석 가능
- 모델에 구애받지 않음 (Model-agnostic)

In [ ]:
# SHAP 임포트
try:
    import shap
    print(f"SHAP 버전: {shap.__version__}")
except ImportError:
    print("SHAP 설치가 필요합니다: pip install shap")

In [ ]:
# SHAP 계산
import shap

# TreeExplainer (Random Forest용)
explainer = shap.TreeExplainer(best_rf)

# 테스트 데이터 일부로 SHAP 계산 (속도 위해)
X_test_sample = X_test_scaled[:100]
shap_values = explainer.shap_values(X_test_sample)

print(f"SHAP values shape: {shap_values[1].shape}")

In [ ]:
# SHAP Summary Plot (Plotly 버전)
shap_df = pd.DataFrame(
    shap_values[1],  # 이탈 클래스에 대한 SHAP
    columns=feature_names
)

# 평균 절대 SHAP 값
mean_abs_shap = shap_df.abs().mean().sort_values(ascending=True)

fig = px.bar(
    x=mean_abs_shap.values,
    y=mean_abs_shap.index,
    orientation='h',
    title='SHAP Feature Importance (평균 |SHAP|)',
    labels={'x': 'Mean |SHAP Value|', 'y': 'Feature'},
    color=mean_abs_shap.values,
    color_continuous_scale='Plasma'
)

fig.update_layout(height=500)
fig.show()

In [ ]:
# SHAP Beeswarm Plot (Plotly 버전)
# 상위 5개 특성만 시각화
top_features = mean_abs_shap.tail(5).index.tolist()

fig = make_subplots(rows=1, cols=5, subplot_titles=top_features)

for i, feat in enumerate(top_features):
    feat_idx = feature_names.index(feat)
    
    fig.add_trace(
        go.Scatter(
            x=shap_values[1][:, feat_idx],
            y=X_test_sample[:, feat_idx],
            mode='markers',
            marker=dict(
                color=X_test_sample[:, feat_idx],
                colorscale='RdBu',
                size=5
            ),
            showlegend=False
        ),
        row=1, col=i+1
    )

fig.update_layout(
    title='SHAP Values vs Feature Values (상위 5개 특성)',
    height=400
)
fig.show()

In [ ]:
# 개별 예측 설명 (Waterfall Plot 대신 Bar Plot)
sample_idx = 0
sample_shap = shap_values[1][sample_idx]

explanation_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP Value': sample_shap,
    'Feature Value': X_test_sample[sample_idx]
}).sort_values('SHAP Value', key=abs, ascending=False)

fig = go.Figure()

colors = ['red' if x > 0 else 'blue' for x in explanation_df['SHAP Value']]

fig.add_trace(go.Bar(
    x=explanation_df['SHAP Value'],
    y=explanation_df['Feature'],
    orientation='h',
    marker_color=colors,
    text=[f'{v:.2f}' for v in explanation_df['SHAP Value']],
    textposition='outside'
))

actual = y_test.iloc[sample_idx]
pred_proba = best_rf.predict_proba(X_test_sample[sample_idx:sample_idx+1])[0][1]

fig.update_layout(
    title=f'개별 예측 설명 (샘플 {sample_idx})<br>실제: {actual}, 예측 확률: {pred_proba:.2%}',
    xaxis_title='SHAP Value (이탈 방향: +, 유지 방향: -)',
    yaxis_title='Feature',
    height=500
)

fig.show()

---

## 2.4 Partial Dependence Plot (PDP)

In [ ]:
# Partial Dependence 계산
from sklearn.inspection import partial_dependence

# 상위 2개 특성에 대해 PDP 계산
top_2_features = mean_abs_shap.tail(2).index.tolist()
top_2_indices = [feature_names.index(f) for f in top_2_features]

pd_results = partial_dependence(
    best_rf, X_train_scaled, features=top_2_indices,
    kind='average', grid_resolution=50
)

print(f"PDP 계산 완료: {top_2_features}")

In [ ]:
# PDP 시각화
fig = make_subplots(rows=1, cols=2, subplot_titles=top_2_features)

for i, (feat, idx) in enumerate(zip(top_2_features, top_2_indices)):
    fig.add_trace(
        go.Scatter(
            x=pd_results['grid_values'][i],
            y=pd_results['average'][i],
            mode='lines',
            name=feat,
            line=dict(width=3)
        ),
        row=1, col=i+1
    )

fig.update_layout(
    title='Partial Dependence Plot (상위 2개 특성)',
    height=400
)
fig.update_yaxes(title_text='Predicted Probability', row=1, col=1)
fig.update_yaxes(title_text='Predicted Probability', row=1, col=2)

fig.show()

In [ ]:
# 2D PDP (두 특성 간 상호작용)
pd_2d = partial_dependence(
    best_rf, X_train_scaled,
    features=[top_2_indices],  # 2D로 조합
    kind='average',
    grid_resolution=20
)

fig = go.Figure(data=go.Heatmap(
    z=pd_2d['average'][0],
    x=pd_2d['grid_values'][0][0],
    y=pd_2d['grid_values'][0][1],
    colorscale='RdBu_r'
))

fig.update_layout(
    title=f'2D Partial Dependence: {top_2_features[0]} vs {top_2_features[1]}',
    xaxis_title=top_2_features[0],
    yaxis_title=top_2_features[1],
    height=500
)

fig.show()

### 실무 예시: 모델 해석 리포트

In [ ]:
# 모델 해석 요약 리포트
print("=" * 60)
print("모델 해석 요약 리포트")
print("=" * 60)

print("\n1. 모델 성능")
print(f"   - AUC: {roc_auc_score(y_test, y_proba_best_rf):.4f}")
print(f"   - Accuracy: {accuracy_score(y_test, y_pred_best_rf):.4f}")

print("\n2. 상위 중요 특성 (SHAP 기반)")
for i, (feat, val) in enumerate(mean_abs_shap.tail(5).items(), 1):
    print(f"   {i}. {feat}: {val:.4f}")

print("\n3. 비즈니스 인사이트")
top_feat = mean_abs_shap.idxmax()
print(f"   - '{top_feat}'이 이탈 예측에 가장 큰 영향")
print(f"   - 해당 특성 관리로 이탈 방지 가능")

print("\n" + "=" * 60)

---

## 실습 퀴즈

**난이도**: (쉬움) ~ (어려움)

---

### Q1. Hard Voting 이해하기 ⭐

**문제**: 3개 모델의 예측이 다음과 같을 때, Hard Voting 결과는?

```python
model_a_pred = 1
model_b_pred = 0
model_c_pred = 1
```

**기대 결과**: 최종 예측 클래스 출력

In [ ]:
model_a_pred = 1
model_b_pred = 0
model_c_pred = 1

# 여기에 코드를 작성하세요


### Q2. Soft Voting 계산하기 ⭐⭐

**문제**: 3개 모델의 클래스 1 예측 확률이 다음과 같을 때, Soft Voting 결과는?

```python
proba_a = 0.8  # 모델 A의 클래스 1 확률
proba_b = 0.3  # 모델 B의 클래스 1 확률
proba_c = 0.6  # 모델 C의 클래스 1 확률
```

**기대 결과**: 평균 확률과 최종 예측 클래스

In [ ]:
proba_a = 0.8
proba_b = 0.3
proba_c = 0.6

# 여기에 코드를 작성하세요


### Q3. VotingClassifier 구성하기 ⭐⭐

**문제**: LogisticRegression과 RandomForest로 Soft Voting 앙상블을 구성하세요.

```python
# Iris 데이터 사용
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, (iris.target == 2).astype(int)  # 이진 분류
```

In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

iris = load_iris()
X, y = iris.data, (iris.target == 2).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 여기에 코드를 작성하세요


### Q4. Stacking vs Voting 비교 ⭐⭐⭐

**문제**: 같은 베이스 모델로 Stacking과 Voting 성능을 비교하세요.

조건:
- 베이스 모델: LR, RF, GB
- Stacking 메타 모델: LogisticRegression
- 위 iris 데이터 사용

In [ ]:
from sklearn.ensemble import StackingClassifier, GradientBoostingClassifier

# 여기에 코드를 작성하세요


### Q5. 가중 Voting 최적화 ⭐⭐⭐

**문제**: 개별 모델 성능에 비례하여 가중치를 설정한 Weighted Voting을 구성하세요.

```python
# 각 모델의 CV 점수
lr_score = 0.85
rf_score = 0.90
gb_score = 0.92
```

In [ ]:
lr_score = 0.85
rf_score = 0.90
gb_score = 0.92

# 여기에 코드를 작성하세요


### Q6. Optuna 기본 사용 ⭐⭐⭐

**문제**: Optuna로 LogisticRegression의 C 파라미터를 최적화하세요.

조건:
- C 범위: 0.01 ~ 100 (log uniform)
- trials: 20회

In [ ]:
import optuna

# 여기에 코드를 작성하세요


### Q7. Feature Importance 비교 ⭐⭐⭐⭐

**문제**: Random Forest와 Gradient Boosting의 Feature Importance를 비교하는 Plotly 차트를 만드세요.

In [ ]:
# 위에서 사용한 데이터 활용
# 여기에 코드를 작성하세요


### Q8. SHAP Force Plot 해석 ⭐⭐⭐⭐

**문제**: 테스트 데이터의 첫 번째 샘플에 대해 SHAP 값을 계산하고 해석하세요.

요구사항:
1. 예측 확률 출력
2. 상위 3개 영향 특성과 SHAP 값 출력
3. 예측에 대한 해석 문장 작성

In [ ]:
# 위에서 계산한 SHAP 값 활용
# 여기에 코드를 작성하세요


### Q9. PDP 그리기 ⭐⭐⭐⭐⭐

**문제**: 가장 중요한 특성 1개에 대한 Partial Dependence Plot을 Plotly로 그리세요.

추가 요구사항:
- 실제 데이터 분포를 rug plot 형태로 하단에 표시

In [ ]:
# 여기에 코드를 작성하세요


### Q10. 종합: 앙상블 + AutoML + 해석 ⭐⭐⭐⭐⭐

**문제**: 다음 파이프라인을 완성하세요.

1. Optuna로 RandomForest 최적화 (10 trials)
2. 최적 RF + LR + GB로 Stacking 구성
3. SHAP로 상위 3개 중요 특성 출력
4. 모델 성능 (AUC) 출력

In [ ]:
# 여기에 코드를 작성하세요


---

## 학습 정리

### Part 1: 고급 앙상블 핵심 요약

| 앙상블 기법 | 특징 | 장점 | 단점 |
|-----------|------|-----|------|
| Hard Voting | 다수결 | 간단, 빠름 | 확률 정보 무시 |
| Soft Voting | 확률 평균 | 더 정교한 결합 | 확률 필요 |
| Stacking | 메타 모델 | 최고 성능 | 과적합 위험 |
| Blending | 홀드아웃 기반 | 구현 간단 | 데이터 손실 |

### Part 2: AutoML & 모델 해석 핵심 요약

| 도구/기법 | 용도 | 핵심 포인트 |
|---------|------|------------|
| Optuna | 하이퍼파라미터 최적화 | 베이지안 최적화, 조기 종료 |
| Feature Importance | 특성 중요도 | 트리 기반 모델에 내장 |
| Permutation Importance | 모델 불가지론적 중요도 | 셔플 후 성능 저하 측정 |
| SHAP | 개별 예측 설명 | 게임 이론 기반, 해석 가능 AI |
| PDP | 특성-예측 관계 | 다른 특성 고정, 한 특성 변화 |

### 실무 팁

1. **Voting 먼저**: Stacking 전에 Voting으로 빠르게 테스트
2. **다양성 확보**: 앙상블은 다른 알고리즘 조합이 효과적
3. **Optuna 시간 관리**: n_trials 대신 timeout 옵션 활용
4. **SHAP 필수**: 비즈니스 설명에 필수 (규제 대응)
5. **PDP로 검증**: Feature Importance와 방향성 확인